In [5]:
import json
import pandas as pd
from nltk.tokenize import sent_tokenize,word_tokenize

In [6]:
with open('data/squad/train-v1.1.json') as json_data:
    train_data = json.load(json_data)
with open('data/squad/dev-v1.1.json') as json_data:
    dev_data = json.load(json_data)
train_data['data'].extend(dev_data['data'])

In [7]:
data = []
all_question = []
full_paragraph= []
paragraph_question = []
paragraph_answer = []
all_answer = []
for data_index, data_paragraph in enumerate(train_data['data']):
    context = []
    question = []
    ans = []
    for each_paragraph in data_paragraph['paragraphs']:
            question_data = each_paragraph['qas']
            context_data = each_paragraph['context']
            context.append(sent_tokenize(context_data))
            for number_of_question in range(len(question_data)):
                question.append(question_data[number_of_question]['question'])
                start_index = question_data[number_of_question]['answers'][0]['answer_start']
                answer = question_data[number_of_question]['answers'][0]['text']
                end_index = start_index + len(answer)
                full_stop = []
                for pos,char in zip(range(len(context_data)),context_data):
                    if char == '.':
                        full_stop.append(pos)
                full_stop.append(start_index)
                full_stop = sorted(full_stop)
                for pos,value in enumerate(full_stop):
                    if value == start_index:
                        if pos == 0:
                            answer_start = pos
                        else:
                            answer_start = full_stop[pos - 1] + 2
                        if pos == len(full_stop) - 1:
                            answer_stop = pos
                        else:
                            answer_stop = full_stop[pos + 1] 
                ans.append(context_data[answer_start:answer_stop])
    paragraph_question.append(question)       
    full_paragraph.append(context)
    paragraph_answer.append(ans)
    context = []
    question = []
    ans = []
data.append(full_paragraph)
all_question.append(paragraph_question)
all_answer.append(paragraph_answer)

In [8]:
sent = []
all_sent = []
for each_paragraph in data[0]:
    for each_context in each_paragraph:
        for each_sent in each_context:
            sent.append(each_sent)
    all_sent.append(sent)
    sent = []
all_ques,ques = [],[]
for paragraph_question in all_question[0]:
    for each_question in paragraph_question:
        ques.append(each_question)
    all_ques.append(ques)
    ques = []
all_ans,ans = [],[]
for paragraph_answer in all_answer[0]:
    for each_answer in paragraph_answer:
        ans.append(each_answer)
    all_ans.append(ans)
    ans = []

In [9]:
ques_ans_sent = pd.DataFrame({'sent':all_sent,'ans':all_ans,'ques':all_ques})

In [ ]:
EMD = []
sentence = []
for sent in all_sent[0]:
    for ques in all_ques[0]:
        wmd_tag = get_wmd_for_tags_test(ques,sent)
        wmd_glove = get_emd(ques,sent)
        EMD.append((wmd_tag,wmd_glove))
        sentence.append(sent)